## Load libraries and data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import re
import regex as re

!pip install gensim
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics.pairwise import cosine_similarity


In [3]:

glove_file = '/content/drive/MyDrive/TEXT MINING/GloVe/glove.6B.100d.txt'

word2vec_temp_file = get_tmpfile("glove_word2vec.txt")
glove2word2vec(glove_file, word2vec_temp_file)

glove_model = KeyedVectors.load_word2vec_format(word2vec_temp_file)

<ipython-input-3-8b2fd2c1a04f>:4: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_file, word2vec_temp_file)


In [ ]:
# Load collection and emotion data

df = pd.read_csv('/content/drive/MyDrive/TEXT MINING/Data/processed_dataset.csv')

emotion_df = pd.read_csv('/content/drive/MyDrive/TEXT MINING/Data/emotional_vectors.csv')

In [ ]:
df.head()

,text,label,mapped_emotion,mapped_sentiment,processed_text
0,Man I love reddit.,love,joy,positive,man love reddit
1,Right? Considering it’s such an important docu...,gratitude,joy,positive,right considering is important document should...
2,that's adorable asf,amusement,joy,positive,is adorable asf
3,"""Seeeee! We have one of them coloureds too!""",excitement,joy,positive,see have one coloureds too
4,"Lots, play store or apple store vpn. Nord is good",admiration,joy,positive,lot play store apple store vpn nord is good


In [ ]:
df['document_tokens'] = df['processed_text'].apply(lambda x: x.split())
df.head()

,text,label,mapped_emotion,mapped_sentiment,processed_text,document_tokens
0,Man I love reddit.,love,joy,positive,man love reddit,"[man, love, reddit]"
1,Right? Considering it’s such an important docu...,gratitude,joy,positive,right considering is important document should...,"[right, considering, is, important, document, ..."
2,that's adorable asf,amusement,joy,positive,is adorable asf,"[is, adorable, asf]"
3,"""Seeeee! We have one of them coloureds too!""",excitement,joy,positive,see have one coloureds too,"[see, have, one, coloureds, too]"
4,"Lots, play store or apple store vpn. Nord is good",admiration,joy,positive,lot play store apple store vpn nord is good,"[lot, play, store, apple, store, vpn, nord, is..."


In [ ]:
emotion_set = set(emotion_df['all_words'])
len(emotion_set)

9446

In [ ]:
X, y = df[['processed_text', 'document_tokens']], df['mapped_emotion']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

## Train Set processing

In [ ]:
X_train

,processed_text,document_tokens
12019,thanks not declaring karma whory way like so m...,"[thanks, not, declaring, karma, whory, way, li..."
13058,guess respect boot ground cop are not one comi...,"[guess, respect, boot, ground, cop, are, not, ..."
2130,what will have dinner,"[what, will, have, dinner]"
3290,like scared actually,"[like, scared, actually]"
9111,surprise like get so moody when am hungry laug...,"[surprise, like, get, so, moody, when, am, hun..."
...,...,...
9037,funny how name only lie when you want too amazing,"[funny, how, name, only, lie, when, you, want,..."
13107,haha love happy will see if is interested,"[haha, love, happy, will, see, if, is, interes..."
10568,awesome pic,"[awesome, pic]"
17661,name least ha pretty face great skin name look...,"[name, least, ha, pretty, face, great, skin, n..."


In [ ]:
# Extract all the tokens in the given collection and create the set of unique tokens in the collection,
# using as input the column of document tokens

def extract_tokens(list_of_documents):

  all_tokens = [token for sublist in list_of_documents for token in sublist]

  return set(all_tokens)

In [ ]:
train_word_set = extract_tokens(X_train['document_tokens'])
len(train_word_set)

11694

In [ ]:
train_emo_set = emotion_set.intersection(train_word_set)
len(train_emo_set)

8514

In [ ]:
# Extract word embeddings of the GloVe model, given the set of unique words and the embedding dimension.
# If the word is not present in the glove model, fill with a zero vector

def embedding_dict(words_set, glove_model, embedding_dim):

  embedding_dict = {}

  for word in words_set:

    if word in glove_model:
      embedding_dict[word] = glove_model[word]
    else:
      embedding_dict[word] = np.zeros(embedding_dim)

  return embedding_dict

In [ ]:
train_temp_dict = embedding_dict(train_emo_set, glove_model, 100)

In [ ]:
len(train_temp_dict)

8514

In [ ]:
train_embedding_matrix = np.array(list(train_temp_dict.values()))
train_embedding_matrix.shape

(8514, 100)

In [ ]:
train_sim_matrix = cosine_similarity(train_embedding_matrix)
train_sim_matrix.shape

(8514, 8514)

In [ ]:
# Construct a dictionary of top M similar words, according with the cosine similarity matrix

def m_sim_words(word_list, similarity_matrix, M):

  sim_words = dict()
  sim_indices = np.argsort(-similarity_matrix, axis = 1)[:, 1:M+1]

  for i, word in enumerate(word_list):

    sim_words[word] = [word_list[j] for j in sim_indices[i]]

  return sim_words

In [ ]:
train_sim_words = m_sim_words(list(train_temp_dict.keys()), train_sim_matrix, 5)

In [ ]:
# Construct a dictionary in which every word embedding is computed using the top M similar words, averaged with it's glove representation

def semantically_similar_features_vectors(m_sim_words, embedding_dict):

  mean_of_m_vectors = dict()

  for word in m_sim_words:
    mean_of_m_vectors[word] = np.mean([embedding_dict[w] for w in m_sim_words[word]], axis = 0)

  fe_dict = dict()

  for word in embedding_dict:
    fe_dict[word] = np.mean((embedding_dict[word], mean_of_m_vectors[word]), axis = 0)

  return fe_dict

In [ ]:
train_SSF_dict = semantically_similar_features_vectors(train_sim_words, train_temp_dict)

In [ ]:
# Create a dictionary with all words (emotion + simple) taking as input the unique words of a given collection and the realtive emotion dictionary

def complete_dict(words, emotion_dict, model, embedding_dim):

  complete_dict = dict()

  for word in words:
    if word in emotion_dict:
      complete_dict[word] = emotion_dict[word]

    elif word in model:
      complete_dict[word] = model[word]

    else:
      complete_dict[word] = np.zeros(embedding_dim)

  return complete_dict

In [ ]:
train_complete_dict = complete_dict(train_word_set, train_SSF_dict, glove_model, 100)
len(train_complete_dict)

11694

## Test Set Processing

In order to avoid data contamination between test_set and train_set words, the previous code is rearranged specifically for the test.

In [ ]:
X_test

,processed_text,document_tokens
9745,am so glad you were not physically hurt,"[am, so, glad, you, were, not, physically, hurt]"
9532,is great editing know took lot work time aweso...,"[is, great, editing, know, took, lot, work, ti..."
12390,bad news dude you are almost year past,"[bad, news, dude, you, are, almost, year, past]"
8727,lox is optional how dare you put name your fla...,"[lox, is, optional, how, dare, you, put, name,..."
17227,can you show example is more imagined victim h...,"[can, you, show, example, is, more, imagined, ..."
...,...,...
3442,oh alright fair enough haha if sabre not pan o...,"[oh, alright, fair, enough, haha, if, sabre, n..."
9989,thanks very cool wa marriage gift so explains ...,"[thanks, very, cool, wa, marriage, gift, so, e..."
16393,oh damn am clemson student swore had flair,"[oh, damn, am, clemson, student, swore, had, f..."
4036,ahh name you have done again,"[ahh, name, you, have, done, again]"


In [ ]:
test_word_set = extract_tokens(X_test['document_tokens'])
len(test_word_set)

5305

In [ ]:
test_emo_set = emotion_set.intersection(test_word_set)
len(test_emo_set)

4156

In [ ]:
test_temp_dict = embedding_dict(test_emo_set, glove_model, 100)
len(test_temp_dict)

4156

In [ ]:
test_embedding_matrix = np.array(list(test_temp_dict.values()))
test_embedding_matrix.shape

(4156, 100)

In [ ]:
# test similarity matrix, different from the train one:

test_sim_matrix = cosine_similarity(test_embedding_matrix, train_embedding_matrix)
test_sim_matrix.shape

(4156, 8514)

In [ ]:
sim_indices = np.argsort(-test_sim_matrix, axis = 1)[:, 0:5]
len(sim_indices)

4156

In [ ]:
len(list(test_temp_dict.keys()))

4156

In [ ]:
sim_indices[:10]

array([[   0, 4364, 2976, 2062, 5705],
       [   1, 6441, 5506, 1012, 1226],
       [   2, 7485, 6723, 1724,  583],
       [6397, 2800, 7262, 2926, 4432],
       [2165, 5071,  902, 2311,  164],
       [4308, 7167, 4802, 5473, 8112],
       [1211, 2699, 2467, 7339, 1423],
       [6811, 3698, 3501, 3076, 3359],
       [4309, 5928, 4039, 8154, 3162],
       [   3,  831, 6922, 4140, 4467]])

In [ ]:
# Construct a link between words in the test set, and the top M similar in the train set

def test_m_sim_words(test_word_list, train_word_list, similarity_matrix, M):

  sim_words = dict()
  sim_indices = np.argsort(-similarity_matrix, axis = 1)[:, 0:M]

  for i, word in enumerate(test_word_list):

    sim_words[word] = [train_word_list[j] for j in sim_indices[i]]

  return sim_words

In [ ]:
# in this case we have to pass as input words taken from the train dictionary, because we want to avoid data contamination between the words inside the test

test_sim_words = test_m_sim_words(list(test_temp_dict.keys()), list(train_temp_dict.keys()), test_sim_matrix, 5)
len(test_sim_words)

4156

In [ ]:
# Construct word embeddings taking the top M similar train_set words, averaged with the corresponding word in the test_set

def test_sem_sim_features_vectors(m_sim_words, train_embedding_dict, test_embedding_dict):

  mean_of_m_vectors = dict()

  for word in m_sim_words:
    mean_of_m_vectors[word] = np.mean([train_embedding_dict[w] for w in m_sim_words[word]], axis = 0)

  fe_dict = dict()

  for word in test_embedding_dict:
    fe_dict[word] = np.mean((test_embedding_dict[word], mean_of_m_vectors[word]), axis = 0)

  return fe_dict

In [ ]:
#also in this case we want to use the embeddings taken from the train dictionary

test_SSF_dict = test_sem_sim_features_vectors(test_sim_words, train_temp_dict, test_temp_dict)
len(test_SSF_dict)

4156

In [ ]:
test_complete_dict = complete_dict(test_word_set, test_SSF_dict, glove_model, 100)
len(test_complete_dict)

5305

## Save Data for Classification

In [ ]:
test_sim_words_df = pd.DataFrame.from_dict(test_sim_words, orient='index', columns = {'top 1': '0','top 2': '1','top 3': '2','top 4': '3','top 5': '4'})
test_sim_words_df

,top 1,top 2,top 3,top 4,top 5
desire,desire,determination,wish,sense,commitment
intelligence,intelligence,information,security,military,secret
doctor,doctor,physician,nurse,patient,medical
mock,mock,elaborate,assault,shoot,simulate
reflect,reflect,indicate,suggest,change,clearly
...,...,...,...,...,...
living,living,live,life,people,working
mouthful,doozy,stinky,whiff,nauseating,blush
thoughtful,thoughtful,insightful,perceptive,honest,passionate
glad,glad,thankful,pleased,happy,definitely


In [ ]:
train_sim_words_df = pd.DataFrame.from_dict(train_sim_words, orient='index', columns = {'top 1': '0','top 2': '1','top 3': '2','top 4': '3','top 5': '4'})
train_sim_words_df

,top 1,top 2,top 3,top 4,top 5
desire,determination,wish,sense,commitment,ambition
intelligence,information,security,military,secret,defense
doctor,physician,nurse,patient,medical,hospital
weirdo,foodie,goofball,cynic,buffoon,asshole
unattractive,unappealing,attractive,enticing,uninteresting,unimportant
...,...,...,...,...,...
unloaded,loaded,carted,towed,cargo,dumped
detained,arrest,suspected,custody,accused,suspect
suite,lounge,room,bedroom,hotel,studio
signature,unique,logo,piece,unusual,similar


In [ ]:
train_complete_df = pd.DataFrame.from_dict(train_complete_dict, orient='index')
train_complete_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
desire,0.334267,0.088607,0.364037,0.316822,-0.107993,0.060112,-0.711484,-0.443872,0.202189,0.205509,...,-0.555954,-0.048236,-0.041990,0.164741,-0.406853,-0.285642,-0.422283,-0.859976,0.118210,0.651965
intelligence,-0.283872,-0.318484,0.606192,-0.186863,-0.004733,-0.482157,-0.351616,-0.415083,0.142384,0.678678,...,0.073257,-0.044164,-0.271742,0.160164,-0.123191,0.215052,0.124766,-0.745349,-0.000682,0.418061
doctor,-0.043615,-0.271802,-0.122581,0.291644,-0.358188,0.662713,0.325866,0.163132,-0.316316,-0.100219,...,0.662207,-0.255278,0.750775,0.308070,-0.256255,0.814269,0.708766,-0.311738,-0.061111,-0.625675
broly,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
weirdo,0.047350,0.126158,0.347327,-0.468976,-0.444759,0.438522,0.049371,-0.425582,0.527911,0.544099,...,0.039848,0.201898,-0.099464,-0.247751,0.230727,-0.289216,0.255578,-0.272573,-0.152600,0.386400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
unloaded,-0.139565,0.182643,0.197040,-0.572000,0.016356,-0.297338,0.153322,0.402754,0.353429,0.027509,...,-0.397137,0.025097,0.271182,-0.496291,-0.103382,-0.092961,0.236092,0.558049,0.383172,0.183588
detained,0.332544,-0.490514,-0.181255,-0.180102,0.619618,0.793602,-0.129231,0.113311,0.161303,0.547570,...,-0.083851,-0.288437,0.263475,-0.385342,0.032499,0.374331,-0.176840,-0.055660,0.244214,-0.449974
suite,-0.098586,0.026525,0.643784,0.601711,0.067274,0.945034,0.592480,0.814454,-0.195682,0.295085,...,0.374500,-0.242284,0.649068,0.096493,0.072082,0.446951,0.032962,-0.386078,0.947612,-0.020455
signature,-0.120900,0.096421,-0.075425,-0.100600,0.690925,0.109834,0.372530,0.031039,-0.297733,0.473211,...,0.034466,0.197232,-0.047477,-0.032692,-0.179452,-0.174607,-0.891850,-0.672267,0.635504,0.237242


In [ ]:
test_complete_df = pd.DataFrame.from_dict(test_complete_dict, orient='index')
test_complete_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
desire,0.332694,0.102095,0.288304,0.343423,-0.133275,0.103637,-0.735626,-0.471968,0.208434,0.108908,...,-0.535376,-0.051541,-0.081485,0.141324,-0.428946,-0.286602,-0.421683,-0.865667,0.110629,0.642251
intelligence,-0.275438,-0.328169,0.623522,-0.201307,0.066506,-0.531564,-0.397011,-0.441886,0.211678,0.729908,...,-0.012647,0.020611,-0.238780,0.142186,-0.111242,0.193441,0.152373,-0.820925,-0.080881,0.453255
doctor,-0.034743,-0.314418,-0.034130,0.269769,-0.319639,0.643612,0.315167,0.089194,-0.356333,-0.125026,...,0.607368,-0.166609,0.758695,0.304448,-0.166925,0.736969,0.661618,-0.335410,-0.156926,-0.652816
dwelling,-0.075457,0.671859,0.178450,0.332778,-0.501510,0.574170,-0.245862,-0.038322,0.469089,0.496474,...,-0.032520,-0.718227,0.570674,-0.085792,-0.072881,-0.543754,-0.135458,-0.341364,0.547215,-0.467066
weirdo,0.031341,0.171659,0.329507,-0.457004,-0.458036,0.458497,0.052444,-0.454664,0.525835,0.565105,...,0.043701,0.172331,-0.074576,-0.230225,0.208737,-0.264507,0.276134,-0.257521,-0.144991,0.404945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
revolutionary,0.380656,0.645021,-0.429304,0.149433,0.521448,0.350707,0.155546,-0.852059,-0.379749,0.592111,...,0.528144,-0.435603,-0.380181,1.024737,-0.833773,-0.863501,-0.215321,-1.068876,-0.029148,0.102964
include,-0.353311,0.438766,0.149168,0.127834,0.963695,0.020258,-0.352414,0.268122,-0.766411,0.028963,...,-0.152090,-0.415267,-0.142401,-0.192898,-0.445606,-0.077292,-0.431883,-0.647209,1.479060,-0.092952
score,-0.036636,0.141154,0.686787,-0.420132,-0.093689,0.703064,0.376653,-0.279186,-1.238624,0.117487,...,-0.709211,0.058919,-0.594874,0.050060,-0.897869,-0.042526,0.014523,-0.028028,-0.113237,-0.011834
living,0.244277,0.732688,0.500513,-0.088569,0.016027,0.637242,-0.465008,0.141737,-0.008369,0.276896,...,0.501387,-0.740355,0.823107,-0.351459,-0.835265,-0.139831,-0.398147,-0.307548,0.814960,-0.471576


In [ ]:
train_sim_words_df.to_csv('/content/drive/MyDrive/TEXT MINING/Data/train_top_5_similarities.csv', index=True)

test_sim_words_df.to_csv('/content/drive/MyDrive/TEXT MINING/Data/test_top_5_similarities.csv', index=True)

In [ ]:
train_complete_df.to_csv('/content/drive/MyDrive/TEXT MINING/Data/train_complete_embeddings.csv', index=True)

test_complete_df.to_csv('/content/drive/MyDrive/TEXT MINING/Data/test_complete_embeddings.csv', index=True)

# Clustering

In [ ]:
# Total unique words inside all the collection

cluster_word_set = extract_tokens(df['document_tokens'])
len(cluster_word_set)

13161

In [ ]:
cluster_emo_set = emotion_set.intersection(cluster_word_set)
len(cluster_emo_set)

9446

In [ ]:
cluster_temp_dict = embedding_dict(cluster_emo_set, glove_model, 100)
len(cluster_temp_dict)

9446

In [ ]:
cluster_embedding_matrix = np.array(list(cluster_temp_dict.values()))
cluster_embedding_matrix.shape

(9446, 100)

In [ ]:
cluster_sim_matrix = cosine_similarity(cluster_embedding_matrix)
cluster_sim_matrix.shape

(9446, 9446)

In [ ]:
cluster_sim_words = m_sim_words(list(cluster_temp_dict.keys()), cluster_sim_matrix, 5)

In [ ]:
len(cluster_sim_words)

9446

In [ ]:
cluster_SSF_dict = semantically_similar_features_vectors(cluster_sim_words, cluster_temp_dict)

cluster_complete_dict = complete_dict(cluster_word_set, cluster_SSF_dict, glove_model, 100)
len(cluster_complete_dict)

13161

### Save data for Clustering

In [ ]:
cluster_complete_df = pd.DataFrame.from_dict(cluster_complete_dict, orient='index')
cluster_complete_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
desire,0.334267,0.088607,0.364037,0.316822,-0.107993,0.060112,-0.711484,-0.443872,0.202189,0.205509,...,-0.555954,-0.048236,-0.041990,0.164741,-0.406853,-0.285642,-0.422283,-0.859976,0.118210,0.651965
intelligence,-0.283872,-0.318484,0.606192,-0.186863,-0.004733,-0.482157,-0.351616,-0.415083,0.142384,0.678678,...,0.073257,-0.044164,-0.271742,0.160164,-0.123191,0.215052,0.124766,-0.745349,-0.000682,0.418061
doctor,-0.134280,-0.294118,-0.070217,0.262558,-0.320190,0.565326,0.345448,0.043520,-0.397203,-0.103818,...,0.642343,-0.210185,0.762760,0.326416,-0.160897,0.748471,0.658017,-0.373324,-0.158870,-0.612314
dwelling,-0.075457,0.671859,0.178450,0.332778,-0.501510,0.574170,-0.245862,-0.038322,0.469089,0.496474,...,-0.032520,-0.718227,0.570674,-0.085792,-0.072881,-0.543754,-0.135458,-0.341364,0.547215,-0.467066
broly,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mouthful,0.165750,0.489472,0.404424,-0.433086,-0.421483,0.319589,-0.229356,0.043723,0.360951,0.336841,...,0.081107,0.274246,-0.437810,-0.238006,0.672319,-0.126614,-0.303090,-0.019227,-0.286048,0.458000
suite,-0.098586,0.026525,0.643784,0.601711,0.067274,0.945034,0.592480,0.814454,-0.195682,0.295085,...,0.374500,-0.242284,0.649068,0.096493,0.072082,0.446951,0.032962,-0.386078,0.947612,-0.020455
detained,0.332544,-0.490514,-0.181255,-0.180102,0.619618,0.793602,-0.129231,0.113311,0.161303,0.547570,...,-0.083851,-0.288437,0.263475,-0.385342,0.032499,0.374331,-0.176840,-0.055660,0.244214,-0.449974
signature,-0.120900,0.096421,-0.075425,-0.100600,0.690925,0.109834,0.372530,0.031039,-0.297733,0.473211,...,0.034466,0.197232,-0.047477,-0.032692,-0.179452,-0.174607,-0.891850,-0.672267,0.635504,0.237242


In [ ]:
cluster_complete_df.to_csv('/content/drive/MyDrive/TEXT MINING/Data/cluster_complete_embeddings.csv', index=True)